<a href="https://colab.research.google.com/github/lockiultra/SCAMT/blob/main/smiles_encode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 5.4 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/Bibyutatsu/FastJTNNpy3.git

Cloning into 'FastJTNNpy3'...
remote: Enumerating objects: 402, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 402 (delta 15), reused 60 (delta 14), pack-reused 341
Receiving objects: 100% (402/402), 520.25 MiB | 15.31 MiB/s, done.
Resolving deltas: 100% (45/45), done.
Updating files: 100% (308/308), done.


In [3]:
from fast_jtnn.jtnn_vae import JTNNVAE
from fast_jtnn.vocab import Vocab
from tqdm import tqdm

import pandas as pd
import torch

In [4]:
with open('./vocab.txt', 'r') as f:
  vocab_list = [smiles for smiles in f.read().split('\n')]

with open('./smiles.txt', 'r') as f:
  smiles_list = list(set([smiles for smiles in f.read().split('\n')]))

1866

In [5]:
vocab = Vocab(vocab_list)

In [6]:
model = JTNNVAE(vocab, 450, 56, 20, 3)
model.load_state_dict(torch.load('./jtvae.pth'))

/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


<All keys matched successfully>

In [7]:
model.jtnn = model.jtnn.cuda()
model.mpn = model.mpn.cuda()

In [8]:
smiles_encode = []

for smiles in tqdm(smiles_list[1810:]):
  smiles_encode_curr = [float(x_i) for x_i in model.encode_from_smiles([smiles])[0].data.cpu()]
  smiles_encode.append(smiles_encode_curr)

100%|██████████| 56/56 [00:04<00:00, 12.04it/s]


In [9]:
encode_df = pd.DataFrame(smiles_encode)

In [34]:
encode_df

,0,1,2,3,4,5,6,7,8,9,...,890,891,892,893,894,895,896,897,898,899
0,0.0,0.0,1.189552,0.0,0.000000,3.332884,2.011087,0.000000,0.574320,0.0,...,0.0,0.000000,0.134300,0.110668,0.377989,0.0,0.185699,0.064060,0.0,0.0
1,0.0,0.0,3.278831,0.0,3.492615,0.571598,1.518118,0.000000,1.635276,0.0,...,0.0,0.000000,0.037921,0.032112,0.118957,0.0,0.049337,0.050452,0.0,0.0
2,0.0,0.0,0.656119,0.0,2.238433,0.306278,0.000000,2.252180,0.790817,0.0,...,0.0,0.000000,0.000000,0.000000,0.054611,0.0,0.037022,0.000000,0.0,0.0
3,0.0,0.0,2.757748,0.0,3.182679,0.000000,0.000000,3.661519,0.743940,0.0,...,0.0,0.000000,0.095889,0.000000,0.001912,0.0,0.003472,0.003339,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.000000,2.599558,0.000000,3.755938,0.511701,0.0,...,0.0,0.013695,0.084925,0.292385,0.015563,0.0,0.000000,0.007176,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1803,0.0,0.0,0.316774,0.0,0.393586,3.041513,2.971371,0.000000,0.567364,0.0,...,0.0,0.000000,0.099850,0.039400,0.094085,0.0,0.097962,0.000000,0.0,0.0
1804,0.0,0.0,1.533867,0.0,0.461846,2.509447,1.861655,3.901769,1.347834,0.0,...,0.0,0.000000,0.033422,0.000000,0.548713,0.0,0.106501,0.055203,0.0,0.0
1805,0.0,0.0,0.000000,0.0,0.000000,2.004037,0.563324,3.588149,0.000000,0.0,...,0.0,0.000000,0.091050,0.023217,0.084158,0.0,0.022201,0.056665,0.0,0.0
1806,0.0,0.0,0.000000,0.0,0.018938,3.101244,0.000000,3.442647,0.097828,0.0,...,0.0,0.000000,0.235519,0.100716,0.052462,0.0,0.058954,0.033413,0.0,0.0


In [35]:
encode_df.to_csv('./smiles_encoded.csv')